In [3]:
#SVM

In [4]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GroupKFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [5]:
data = pd.read_csv('C:/Users/zhanghui/Desktop/PROJECT/Frogs_MFCCs.csv')
data.head(5)

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [6]:
X_test = np.array(data.ix[:,:22])
y_test = np.array(data['Family'])
np.unique(y_test)

array(['Bufonidae', 'Dendrobatidae', 'Hylidae', 'Leptodactylidae'], dtype=object)

In [7]:
#crossvalidation
groups = np.array(data['RecordID'])
gkf = GroupKFold(n_splits=60)
gkf.split(X_test, y_test, groups)

#SVM on anural
svr = SVC(probability=True, decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, cv=list(gkf.split(X_test, y_test, groups)))

clf.fit(X_test, y_test)

predicted = cross_val_predict(clf.best_estimator_, 
                              X_test, 
                              y_test, 
                              cv=list(gkf.split(X_test, y_test, groups)))


In [8]:
np.unique(y_test)

array(['Bufonidae', 'Dendrobatidae', 'Hylidae', 'Leptodactylidae'], dtype=object)

In [9]:
#matrix on family level
result = np.c_[y_test, predicted]

Bufonidae = 0
Dendrobatidae = 0
Hylidae = 0
Leptodactylidae = 0
total = 0

for i in range(len(y_test)):
    if result[:, 0][i] == 'Leptodactylidae':
        total += 1
        if result[:, 1][i] == 'Bufonidae':
            Bufonidae += 1
        if result[:, 1][i] == 'Dendrobatidae':
            Dendrobatidae += 1
        if result[:, 1][i] == 'Hylidae':
            Hylidae += 1
        if result[:, 1][i] == 'Leptodactylidae':
            Leptodactylidae += 1


Bufonidae, Dendrobatidae, Hylidae, Leptodactylidae, total

(230, 49, 241, 3900, 4420)

In [14]:
index = list()
for i in range(len(predicted)):
    if predicted[i] == 'Hylidae':
        index.append(i)
                
for i in range(4):
    print(clf.best_estimator_.predict_proba(X_test)[index,i].mean())
    
for i in range(4):
    print(clf.best_estimator_.decision_function(X_test)[index,i].mean())

0.00321391512824
0.022353292511
0.873865133831
0.10056765853
0.851564341355
0.455258438315
2.97467943766
1.71849778267


In [15]:
data = np.array(data)
data.shape

(7195, 26)

In [16]:
X_Leptodactylidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Leptodactylidae':
        X_Leptodactylidae.append(data[i])        
X_Leptodactylidae = np.array(X_Leptodactylidae)
y_Leptodactylidae = np.array(X_Leptodactylidae[:,23])

X_Dendrobatidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Dendrobatidae':
        X_Dendrobatidae.append(data[i])        
X_Dendrobatidae = np.array(X_Dendrobatidae)
y_Dendrobatidae = np.array(X_Dendrobatidae[:,23])

X_Hylidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Hylidae':
        X_Hylidae.append(data[i])
X_Hylidae = np.array(X_Hylidae)
y_Hylidae = np.array(X_Hylidae[:,23])

X_Bufonidae = list()
for i in range(len(predicted)):
    if predicted[i] == 'Bufonidae':
        X_Bufonidae.append(data[i])        
X_Bufonidae = np.array(X_Bufonidae)
y_Bufonidae = np.array(X_Bufonidae[:,23])


In [18]:
#data prepare
print(np.unique(X_Bufonidae[:,23]))
X_Bufonidae_test = X_Bufonidae[:,:22]

#group prepare
print(np.unique(X_Bufonidae[:,25]).size)
groups_Bufonidae = np.array(X_Bufonidae[:,25])

['Adenomera' 'Ameerega' 'Leptodactylus' 'Osteocephalus' 'Rhinella' 'Scinax']
15


In [20]:
#crossvalidation
gkf = GroupKFold(n_splits=15)
gkf.split(X_Bufonidae_test, y_Bufonidae, groups_Bufonidae)

#SVM on genus family
svr = SVC(probability=True,decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, cv=list(gkf.split(X_Bufonidae_test, y_Bufonidae, groups_Bufonidae)))


clf.fit(X_Bufonidae_test, y_Bufonidae)

predicted_Bufonidae = cross_val_predict(clf.best_estimator_, 
                                        X_Bufonidae_test, 
                                        y_Bufonidae, 
                                        cv=list(gkf.split(X_Bufonidae_test, y_Bufonidae, groups_Bufonidae)))

In [23]:
index = list()
for i in range(len(predicted_Bufonidae)):
    if predicted_Bufonidae[i] == 'Rhinella':
        index.append(i)
                
for i in range(6):
    print(clf.best_estimator_.predict_proba(X_Bufonidae_test)[index,i].mean())

print('\n')

for i in range(6):
    print(clf.best_estimator_.decision_function(X_Bufonidae_test)[index,i].mean())

0.0459446906964
0.527830053058
0.0057380290927
0.0463583072264
0.312780047562
0.0613488723644


2.79689311425
3.57827536504
-0.130990978673
2.7867378866
4.53560403013
1.43348058266


In [24]:
np.unique(predicted_Bufonidae)

array(['Adenomera', 'Ameerega', 'Leptodactylus', 'Osteocephalus',
       'Rhinella'], dtype=object)

In [31]:
#data prepare
print(np.unique(X_Hylidae[:,23]))
X_Hylidae_test = X_Hylidae[:,:22]

#group prepare
print(np.unique(X_Hylidae[:,25]).size)
groups_Hylidae = np.array(X_Hylidae[:,25])

['Adenomera' 'Ameerega' 'Dendropsophus' 'Hypsiboas' 'Leptodactylus'
 'Osteocephalus' 'Rhinella' 'Scinax']
46


In [33]:
#crossvalidation
gkf = GroupKFold(n_splits=46)
gkf.split(X_Hylidae_test, y_Hylidae, groups_Hylidae)

#SVM on genus family
svr = SVC(probability=True,decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Hylidae_test, 
                                     y_Hylidae, 
                                     groups_Hylidae)))


clf.fit(X_Hylidae_test, y_Hylidae)

predicted_Hylidae = cross_val_predict(clf.best_estimator_, 
                                        X_Hylidae_test, 
                                        y_Hylidae, 
                                        cv=list(gkf.split(X_Hylidae_test, y_Hylidae, groups_Hylidae)))


In [34]:
clf.best_estimator_.decision_function(X_Hylidae_test).shape

(2321, 8)

In [38]:
index = list()
for i in range(len(predicted_Hylidae)):
    if predicted_Hylidae[i] == 'Hypsiboas':
        index.append(i)
                
for i in range(8):
    print(clf.best_estimator_.predict_proba(X_Hylidae_test)[index,i].mean())

print('\n')

for i in range(8):
    print(clf.best_estimator_.decision_function(X_Hylidae_test)[index,i].mean())


0.00436410480264
0.00164159604705
0.00403644995798
0.952768072542
0.0137022325494
0.0166648366947
0.000957549995405
0.00586515741092


3.38315564959
3.10189143139
2.4370085443
7.15632378619
3.03637811915
3.60671610623
1.19642547884
4.0821008843


In [39]:
#data prepare
print(np.unique(X_Dendrobatidae[:,23]))
X_Dendrobatidae_test = X_Dendrobatidae[:,:22]

#group prepare
print(np.unique(X_Dendrobatidae[:,25]).size)
groups_Dendrobatidae = np.array(X_Dendrobatidae[:,25])

['Adenomera' 'Ameerega' 'Dendropsophus']
13


In [40]:
#crossvalidation
gkf = GroupKFold(n_splits=13)
gkf.split(X_Dendrobatidae_test, y_Dendrobatidae, groups_Dendrobatidae)

#SVM on genus family
svr = SVC(probability=True,decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Dendrobatidae_test, 
                                     y_Dendrobatidae, 
                                     groups_Dendrobatidae)))


clf.fit(X_Dendrobatidae_test, y_Dendrobatidae)

predicted_Dendrobatidae = cross_val_predict(clf.best_estimator_, 
                                        X_Dendrobatidae_test, 
                                        y_Dendrobatidae, 
                                        cv=list(gkf.split(X_Dendrobatidae_test, y_Dendrobatidae, groups_Dendrobatidae)))
for i in range(3):
    print(clf.best_estimator_.predict_proba(X_Dendrobatidae_test)[:,i].mean())

0.144591896099
0.809784362365
0.0456237415367


In [41]:
index = list()
for i in range(len(predicted_Dendrobatidae)):
    if predicted_Dendrobatidae[i] == 'Ameerega':
        index.append(i)
                
for i in range(3):
    print(clf.best_estimator_.predict_proba(X_Dendrobatidae_test)[index,i].mean())

print('\n')

for i in range(3):
    print(clf.best_estimator_.decision_function(X_Dendrobatidae_test)[index,i].mean())

0.144591896099
0.809784362365
0.0456237415367


0.95998854101
2.38163639054
-0.341624931549


In [43]:
#data prepare
print(np.unique(X_Leptodactylidae[:,23]))
X_Leptodactylidae_test = X_Leptodactylidae[:,:22]

#group prepare
print(np.unique(X_Leptodactylidae[:,25]).size)
groups_Leptodactylidae = np.array(X_Leptodactylidae[:,25])

['Adenomera' 'Ameerega' 'Dendropsophus' 'Hypsiboas' 'Leptodactylus'
 'Osteocephalus' 'Rhinella' 'Scinax']
44


In [44]:
#crossvalidation
gkf = GroupKFold(n_splits=44)
gkf.split(X_Leptodactylidae_test, y_Leptodactylidae, groups_Leptodactylidae)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Leptodactylidae_test, 
                                     y_Leptodactylidae, 
                                     groups_Leptodactylidae)))


clf.fit(X_Leptodactylidae_test, y_Leptodactylidae)

predicted_Leptodactylidae = cross_val_predict(clf.best_estimator_, 
                                        X_Leptodactylidae_test, 
                                        y_Leptodactylidae, 
                                        cv=list(gkf.split(X_Leptodactylidae_test, y_Leptodactylidae, groups_Leptodactylidae)))

for i in range(8):
    print(clf.best_estimator_.predict_proba(X_Leptodactylidae_test)[:,i].mean())

0.900529105248
0.0229502755159
0.0132595983418
0.00896731962101
0.0492216823979
0.000637241822166
0.00271000718612
0.00172476986721


In [46]:
index = list()
for i in range(len(predicted_Leptodactylidae)):
    if predicted_Leptodactylidae[i] == 'Adenomera':
        index.append(i)
                
for i in range(8):
    print(clf.best_estimator_.predict_proba(X_Leptodactylidae_test)[index,i].mean())

print('\n')

for i in range(8):
    print(clf.best_estimator_.decision_function(X_Leptodactylidae_test)[index,i].mean())

0.960484079807
0.0217819516865
0.0135426747666
0.00192110760237
0.000805264521927
0.000320464815272
0.000634438820293
0.000510017980059


7.37626399878
3.49679796301
6.09596040526
4.0396927469
4.58725604629
-0.285062880869
1.8560570863
0.833034634327


In [47]:
np.unique(predicted_Bufonidae)

array(['Adenomera', 'Ameerega', 'Leptodactylus', 'Osteocephalus',
       'Rhinella'], dtype=object)

In [48]:
np.unique(predicted_Hylidae)

array(['Adenomera', 'Ameerega', 'Dendropsophus', 'Hypsiboas',
       'Leptodactylus', 'Osteocephalus', 'Rhinella', 'Scinax'], dtype=object)

In [49]:
np.unique(predicted_Dendrobatidae)

array(['Ameerega'], dtype=object)

In [50]:
np.unique(predicted_Leptodactylidae)

array(['Adenomera', 'Ameerega', 'Hypsiboas', 'Leptodactylus'], dtype=object)

In [214]:
#matrix on Genus level
result_genus = np.c_[y_Dendrobatidae, predicted_Dendrobatidae]

Adenomera = 0
Ameerega = 0
Dendropsophus = 0
Hypsiboas = 0
Leptodactylus = 0
Osteocephalus = 0
Rhinella = 0
Scinax = 0
total = 0

for i in range(len(y_Dendrobatidae)):
    if result_genus[:, 0][i] == 'Adenomera':
        if result_genus[:, 1][i] == 'Adenomera':
            Adenomera += 1
        if result_genus[:, 1][i] == 'Ameerega':
            Ameerega += 1
        if result_genus[:, 1][i] == 'Dendropsophus':
            Dendropsophus += 1
        if result_genus[:, 1][i] == 'Hypsiboas':
            Hypsiboas += 1
        if result_genus[:, 1][i] == 'Leptodactylus':
            Leptodactylus += 1
        if result_genus[:, 1][i] == 'Osteocephalus':
            Osteocephalus += 1
        if result_genus[:, 1][i] == 'Rhinella':
            Rhinella += 1
        if result_genus[:, 1][i] == 'Scinax':
            Scinax += 1

Adenomera, Ameerega, Dendropsophus, Hypsiboas, Leptodactylus, Osteocephalus, Rhinella, Scinax, total

(0, 49, 0, 0, 0, 0, 0, 0, 0)

In [51]:
#To copy the data predicted to be Adenomera
X_Adenomera = list()
y_Adenomera = list()
for i in range(len(predicted_Leptodactylidae)):
    if predicted_Leptodactylidae[i] == 'Adenomera':
        X_Adenomera.append(X_Leptodactylidae[i])
        
X_Adenomera = np.array(X_Adenomera)
y_Adenomera = np.array(X_Adenomera[:,24])

#To copy the data predicted to be Leptodactylus
X_Leptodactylus = list()
y_Leptodactylus = list()
for i in range(len(predicted_Leptodactylidae)):
    if predicted_Leptodactylidae[i] == 'Leptodactylus':
        X_Leptodactylus.append(X_Leptodactylidae[i])
        
X_Leptodactylus = np.array(X_Leptodactylus)
y_Leptodactylus = np.array(X_Leptodactylus[:,24])

#To copy the data predicted to be Scinax 
X_Scinax  = list()
y_Scinax  = list()
for i in range(len(predicted_Hylidae )):
    if predicted_Hylidae[i] == 'Scinax':
        X_Scinax.append(X_Hylidae[i])

        
X_Scinax = np.array(X_Scinax)
y_Scinax = np.array(X_Scinax[:,24])


#To copy the data predicted to be Hypsiboas
X_Hypsiboas = list()
y_Hypsiboas = list()
for i in range(len(predicted_Hylidae)):
    if predicted_Hylidae[i] == 'Hypsiboas':
        X_Hypsiboas.append(X_Hylidae[i])
        
X_Hypsiboas = np.array(X_Hypsiboas)
y_Hypsiboas = np.array(X_Hypsiboas[:,24])


#To copy the data predicted to be Osteocephalus
X_Osteocephalus = list()
y_Osteocephalus = list()
for i in range(len(predicted_Hylidae)):
    if predicted_Hylidae[i] == 'Osteocephalus':
        X_Osteocephalus.append(X_Hylidae[i])
        
X_Osteocephalus = np.array(X_Osteocephalus)
y_Osteocephalus = np.array(X_Osteocephalus[:,24])


#To copy the data predicted to be Dendropsophus
X_Dendropsophus = list()
y_Dendropsophus = list()
for i in range(len(predicted_Hylidae)):
    if predicted_Hylidae[i] == 'Dendropsophus':
        X_Dendropsophus.append(X_Hylidae[i])
        
X_Dendropsophus = np.array(X_Dendropsophus)
y_Dendropsophus = np.array(X_Dendropsophus[:,24])


#To copy the data predicted to be Rhinella
X_Rhinella = list()
y_Rhinella = list()
for i in range(len(predicted_Bufonidae  )):
    if predicted_Bufonidae  [i] == 'Rhinella':
        X_Rhinella.append(X_Bufonidae[i])
        
X_Rhinella = np.array(X_Rhinella)
y_Rhinella = np.array(X_Rhinella[:,24])


#To copy the data predicted to be Ameerega
X_Ameerega = list()
y_Ameerega = list()
for i in range(len(predicted_Dendrobatidae)):
    if predicted_Dendrobatidae[i] == 'Ameerega':
        X_Ameerega.append(X_Dendrobatidae[i])
        
X_Ameerega = np.array(X_Ameerega)
y_Ameerega = np.array(X_Ameerega[:,24])

In [53]:
#data prepare
print(np.unique(X_Ameerega[:,24]))
X_Ameerega_test = X_Ameerega[:,:22]

#group prepare
print(np.unique(X_Ameerega[:,25]).size)
groups_Ameerega = np.array(X_Ameerega[:,25])

['AdenomeraAndre' 'Ameeregatrivittata' 'HylaMinuta']
13


In [54]:
#crossvalidation
gkf = GroupKFold(n_splits=13)
gkf.split(X_Ameerega_test, y_Ameerega, groups_Ameerega)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Ameerega_test, 
                                     y_Ameerega, 
                                     groups_Ameerega)))


clf.fit(X_Ameerega_test, y_Ameerega)

predicted_Ameerega = cross_val_predict(clf.best_estimator_, 
                                        X_Ameerega_test, 
                                        y_Ameerega, 
                                        cv=list(gkf.split(X_Ameerega_test, y_Ameerega, groups_Ameerega)))

for i in range(3):
    print(clf.best_estimator_.predict_proba(X_Ameerega_test)[:,i].mean())

0.144079409796
0.808489891273
0.0474306989307


In [55]:
index = list()
for i in range(len(predicted_Ameerega)):
    if predicted_Ameerega[i] == 'Ameeregatrivittata':
        index.append(i)
                
for i in range(3):
    print(clf.best_estimator_.predict_proba(X_Ameerega_test)[index,i].mean())

print('\n')

for i in range(3):
    print(clf.best_estimator_.decision_function(X_Ameerega_test)[index,i].mean())

0.144079409796
0.808489891273
0.0474306989307


0.95998854101
2.38163639054
-0.341624931549


In [56]:
#data prepare
print(np.unique(X_Rhinella[:,24]))
X_Rhinella_test = X_Rhinella[:,:22]

#group prepare
print(np.unique(X_Rhinella[:,25]).size)
groups_Rhinella = np.array(X_Rhinella[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'OsteocephalusOophagus' 'Rhinellagranulosa' 'ScinaxRuber']
12


In [57]:
#crossvalidation
gkf = GroupKFold(n_splits=12)
gkf.split(X_Rhinella_test, y_Rhinella, groups_Rhinella)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Rhinella_test, 
                                     y_Rhinella, 
                                     groups_Rhinella)))


clf.fit(X_Ameerega_test, y_Ameerega)

predicted_Rhinella = cross_val_predict(clf.best_estimator_, 
                                    X_Rhinella_test, 
                                    y_Rhinella, 
                                    cv=list(gkf.split(X_Rhinella_test, 
                                                        y_Rhinella, 
                                                        groups_Rhinella)))

for i in range(3):
    print(clf.best_estimator_.predict_proba(X_Rhinella_test)[:,i].mean())

0.00440904958135
0.977944657785
0.0176462926335


In [59]:
index = list()
for i in range(len(predicted_Rhinella)):
    if predicted_Rhinella[i] == 'Rhinellagranulosa':
        index.append(i)
                
for i in range(3):
    print(clf.best_estimator_.predict_proba(X_Rhinella_test)[index,i].mean())

print('\n')

for i in range(3):
    print(clf.best_estimator_.decision_function(X_Rhinella_test)[index,i].mean())

0.00440904958135
0.977944657785
0.0176462926335


0.834921052791
2.35121597879
-0.186137031585


In [61]:
#data prepare
print(np.unique(X_Dendropsophus[:,24]))
X_Dendropsophus_test = X_Dendropsophus[:,:22]

#group prepare
print(np.unique(X_Dendropsophus[:,25]).size)
groups_Dendropsophus = np.array(X_Dendropsophus[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'HypsiboasCordobae' 'LeptodactylusFuscus' 'ScinaxRuber']
22


In [62]:
#crossvalidation
gkf = GroupKFold(n_splits=22)
gkf.split(X_Dendropsophus_test, y_Dendropsophus, groups_Dendropsophus)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape='ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Dendropsophus_test, 
                                     y_Dendropsophus, 
                                     groups_Dendropsophus)))


clf.fit(X_Dendropsophus_test, y_Dendropsophus)

predicted_Dendropsophus = cross_val_predict(clf.best_estimator_, 
                                    X_Dendropsophus_test, 
                                    y_Dendropsophus, 
                                    cv=list(gkf.split(X_Dendropsophus_test, 
                                                        y_Dendropsophus, 
                                                        groups_Dendropsophus)))

for i in range(7):
    print(clf.best_estimator_.predict_proba(X_Dendropsophus_test)[:,i].mean())

0.047979176523
0.0289562399142
0.165539140769
0.699094643777
0.00857811484001
0.0418854660147
0.00796721816234


In [63]:
index = list()
for i in range(len(predicted_Dendropsophus)):
    if predicted_Dendropsophus[i] == 'HylaMinuta':
        index.append(i)
                
for i in range(7):
    print(clf.best_estimator_.predict_proba(X_Dendropsophus_test)[index,i].mean())

print('\n')

for i in range(7):
    print(clf.best_estimator_.decision_function(X_Dendropsophus_test)[index,i].mean())

0.047979176523
0.0289562399142
0.165539140769
0.699094643777
0.00857811484001
0.0418854660147
0.00796721816234


3.14616103333
2.24996754801
5.34731880649
6.27542145041
0.117953211337
3.61171327554
0.251464674874


In [64]:
#data prepare
print(np.unique(X_Osteocephalus[:,24]))
X_Osteocephalus_test = X_Osteocephalus[:,:22]

#group prepare
print(np.unique(X_Osteocephalus[:,25]).size)
groups_Osteocephalus = np.array(X_Osteocephalus[:,25])

['AdenomeraAndre' 'HypsiboasCinerascens' 'HypsiboasCordobae'
 'OsteocephalusOophagus']
8


In [66]:
#crossvalidation
gkf = GroupKFold(n_splits=8)
gkf.split(X_Osteocephalus_test, y_Osteocephalus, groups_Osteocephalus)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape = 'ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Osteocephalus_test, 
                                     y_Osteocephalus, 
                                     groups_Osteocephalus)))


clf.fit(X_Osteocephalus_test, y_Osteocephalus)

predicted_Osteocephalus = cross_val_predict(clf.best_estimator_, 
                                    X_Osteocephalus_test, 
                                    y_Osteocephalus, 
                                    cv=list(gkf.split(X_Osteocephalus_test, 
                                                        y_Osteocephalus, 
                                                        groups_Osteocephalus)))

for i in range(4):
    print(clf.best_estimator_.predict_proba(X_Osteocephalus_test)[:,i].mean())

0.754749236666
0.0562255308173
0.0114900423688
0.177535190148


In [67]:
index = list()
for i in range(len(predicted_Osteocephalus)):
    if predicted_Osteocephalus[i] == 'OsteocephalusOophagus':
        index.append(i)
                
for i in range(4):
    print(clf.best_estimator_.predict_proba(X_Osteocephalus_test)[index,i].mean())

print('\n')

for i in range(4):
    print(clf.best_estimator_.decision_function(X_Osteocephalus_test)[index,i].mean())

0.776292967593
0.0276758353571
0.00913678764125
0.186894409409


2.97307701234
0.970955656737
-0.250153280862
2.30612061178


In [68]:
#data prepare
print(np.unique(X_Hypsiboas[:,24]))
X_Hypsiboas_test = X_Hypsiboas[:,:22]

#group prepare
print(np.unique(X_Hypsiboas[:,25]).size)
groups_Hypsiboas = np.array(X_Hypsiboas[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'HypsiboasCinerascens' 'HypsiboasCordobae'
 'LeptodactylusFuscus' 'OsteocephalusOophagus' 'Rhinellagranulosa'
 'ScinaxRuber']
24


In [69]:
#crossvalidation
gkf = GroupKFold(n_splits=24)
gkf.split(X_Hypsiboas_test, y_Hypsiboas, groups_Hypsiboas)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape = 'ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Hypsiboas_test, 
                                     y_Hypsiboas, 
                                     groups_Hypsiboas)))


clf.fit(X_Hypsiboas_test, y_Hypsiboas)

predicted_Hypsiboas = cross_val_predict(clf.best_estimator_, 
                                    X_Hypsiboas_test, 
                                    y_Hypsiboas, 
                                    cv=list(gkf.split(X_Hypsiboas_test, 
                                                        y_Hypsiboas, 
                                                        groups_Hypsiboas)))

for i in range(10):
    print(clf.best_estimator_.predict_proba(X_Hypsiboas_test)[:,i].mean())

0.00242610370318
0.00174905501626
0.00112804040474
0.00332978324845
0.265879472482
0.685040735045
0.016091271571
0.0177961096324
0.00113980031231
0.00541962858496


In [71]:
index = list()
for i in range(len(predicted_Hypsiboas)):
    if predicted_Hypsiboas[i] == 'HypsiboasCordobae':
        index.append(i)
                
for i in range(10):
    print(clf.best_estimator_.predict_proba(X_Hypsiboas_test)[index,i].mean())

print('\n')

for i in range(10):
    print(clf.best_estimator_.decision_function(X_Hypsiboas_test)[index,i].mean())

0.0010678708809
0.00156992641139
0.000941971213147
0.00395962168819
0.00567583583815
0.95533419232
0.0174523743872
0.00806228155795
0.000931517579565
0.00500440812356


1.72430964871
0.272800545356
5.28411038131
2.18095629339
4.33440646862
9.19514692411
6.83406604434
4.94839789376
3.30250812591
6.92329767448


In [72]:
#data prepare
print(np.unique(X_Scinax[:,24]))
X_Scinax_test = X_Scinax[:,:22]

#group prepare
print(np.unique(X_Scinax[:,25]).size)
groups_Scinax = np.array(X_Scinax[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'HypsiboasCordobae'
 'ScinaxRuber']
8


In [73]:
#crossvalidation
gkf = GroupKFold(n_splits=8)
gkf.split(X_Scinax_test, y_Scinax, groups_Scinax)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape = 'ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Scinax_test, 
                                     y_Scinax, 
                                     groups_Scinax)))


clf.fit(X_Scinax_test, y_Scinax)

predicted_Scinax = cross_val_predict(clf.best_estimator_, 
                                    X_Scinax_test, 
                                    y_Scinax, 
                                    cv=list(gkf.split(X_Scinax_test, 
                                                        y_Scinax, 
                                                        groups_Scinax)))

for i in range(4):
    print(clf.best_estimator_.predict_proba(X_Scinax_test)[:,i].mean())

0.0170333212087
0.0180374100523
0.0158720302012
0.949057238538


In [74]:
index = list()
for i in range(len(predicted_Scinax)):
    if predicted_Scinax[i] == 'ScinaxRuber':
        index.append(i)
                
for i in range(4):
    print(clf.best_estimator_.predict_proba(X_Scinax_test)[index,i].mean())

print('\n')

for i in range(4):
    print(clf.best_estimator_.decision_function(X_Scinax_test)[index,i].mean())

0.0170333212087
0.0180374100523
0.0158720302012
0.949057238538


-0.156284651003
0.987663687521
1.67360756155
3.49501340193


In [75]:
#data prepare
print(np.unique(X_Adenomera[:,24]))
X_Adenomera_test = X_Adenomera[:,:22]

#group prepare
print(np.unique(X_Adenomera[:,25]).size)
groups_Adenomera = np.array(X_Adenomera[:,25])

['AdenomeraAndre' 'AdenomeraHylaedactylus' 'Ameeregatrivittata'
 'HylaMinuta' 'HypsiboasCinerascens' 'HypsiboasCordobae'
 'LeptodactylusFuscus' 'Rhinellagranulosa']
34


In [76]:
#crossvalidation
gkf = GroupKFold(n_splits=34)
gkf.split(X_Adenomera_test, y_Adenomera, groups_Adenomera)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape = 'ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Adenomera_test, 
                                     y_Adenomera, 
                                     groups_Adenomera)))


clf.fit(X_Adenomera_test, y_Adenomera)

predicted_Adenomera = cross_val_predict(clf.best_estimator_, 
                                    X_Adenomera_test, 
                                    y_Adenomera, 
                                    cv=list(gkf.split(X_Adenomera_test, 
                                                        y_Adenomera, 
                                                        groups_Adenomera)))

for i in range(8):
    print(clf.best_estimator_.predict_proba(X_Adenomera_test)[:,i].mean())

0.0592684814703
0.900197999238
0.0209427558492
0.0165673088236
0.000410625171243
0.00173731614369
0.000451674963651
0.000423838340351


In [78]:
index = list()
for i in range(len(predicted_Adenomera)):
    if predicted_Adenomera[i] == 'AdenomeraHylaedactylus':
        index.append(i)
                
for i in range(8):
    print(clf.best_estimator_.predict_proba(X_Adenomera_test)[index,i].mean())

print('\n')

for i in range(8):
    print(clf.best_estimator_.decision_function(X_Adenomera_test)[index,i].mean())

8.95798931485e-05
0.993180970496
0.00021133816436
0.00530810965595
0.000144325801375
0.000767722962782
0.000171680925334
0.000126272101165


0.88424204956
7.18356438531
2.44105304476
6.05569786874
3.23880585867
4.98390765708
3.29868157316
-0.0859524372777


In [79]:
#data prepare
print(np.unique(X_Leptodactylus[:,24]))
X_Leptodactylus_test = X_Leptodactylus[:,:22]

#group prepare
print(np.unique(X_Leptodactylus[:,25]).size)
groups_Leptodactylus = np.array(X_Leptodactylus[:,25])

['AdenomeraAndre' 'Ameeregatrivittata' 'HypsiboasCinerascens'
 'HypsiboasCordobae' 'LeptodactylusFuscus' 'Rhinellagranulosa'
 'ScinaxRuber']
13


In [80]:
#crossvalidation
gkf = GroupKFold(n_splits=13)
gkf.split(X_Leptodactylus_test, y_Leptodactylus, groups_Leptodactylus)

#SVM on genus family
svr = SVC(probability=True, decision_function_shape = 'ovr')

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]}, 
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(estimator=svr, 
                   param_grid=tuned_parameters, 
                   cv=list(gkf.split(X_Leptodactylus_test, 
                                     y_Leptodactylus, 
                                     groups_Leptodactylus)))


clf.fit(X_Leptodactylus_test, y_Leptodactylus)

predicted_Leptodactylus = cross_val_predict(clf.best_estimator_, 
                                    X_Leptodactylus_test, 
                                    y_Leptodactylus, 
                                    cv=list(gkf.split(X_Leptodactylus_test, 
                                                        y_Leptodactylus, 
                                                        groups_Leptodactylus)))


In [83]:
index = list()
for i in range(len(predicted_Leptodactylus)):
    if predicted_Leptodactylus[i] == 'LeptodactylusFuscus':
        index.append(i)
        
for i in range(7):
    print(clf.best_estimator_.predict_proba(X_Leptodactylus_test)[index,i].mean())
    
print('\n')
    
for i in range(7):
    print(clf.best_estimator_.decision_function(X_Leptodactylus_test)[index,i].mean())

0.0088127773437
0.029600412897
0.0421058071197
0.0250084997897
0.853939740399
0.0298651228418
0.0106676396088


1.79819484251
0.273277196675
4.07708416061
3.19696394801
6.2528105181
4.40676087106
0.994908463034


In [82]:
clf.best_estimator_.decision_function(X_Leptodactylus_test).shape

(243, 7)

In [227]:
confidence_LeptodactylusFuscus = list()
for i in range(len(y_Leptodactylus)):
    if predicted_Leptodactylus[i] == 'LeptodactylusFuscus':
        confidence_LeptodactylusFuscus.append(X_Leptodactylus_test[i,:])
        

In [212]:
#matrix on Genus level
result_species = np.c_[y_Ameerega, predicted_Ameerega]

AdenomeraAndre = 0
AdenomeraHylaedactylus = 0
Ameeregatrivittata = 0
HylaMinuta = 0
HypsiboasCinerascens = 0
HypsiboasCordobae = 0
LeptodactylusFuscus = 0
OsteocephalusOophagus = 0
Rhinellagranulosa = 0
ScinaxRuber = 0
total = 0

for i in range(len(y_Ameerega)):
    if result_species[:, 0][i] == 'AdenomeraAndre':
        total += 1
        if result_species[:, 1][i] == 'AdenomeraAndre':
            AdenomeraAndre += 1
        if result_species[:, 1][i] == 'AdenomeraHylaedactylus':
            AdenomeraHylaedactylus += 1
        if result_species[:, 1][i] == 'Ameeregatrivittata':
            Ameeregatrivittata += 1
        if result_species[:, 1][i] == 'HylaMinuta':
            HylaMinuta += 1
        if result_species[:, 1][i] == 'HypsiboasCinerascens':
            HypsiboasCinerascens += 1
        if result_species[:, 1][i] == 'HypsiboasCordobae':
            HypsiboasCordobae += 1
        if result_species[:, 1][i] == 'LeptodactylusFuscus':
            LeptodactylusFuscus += 1
        if result_species[:, 1][i] == 'OsteocephalusOophagus':
            OsteocephalusOophagus += 1
        if result_species[:, 1][i] == 'Rhinellagranulosa':
            Rhinellagranulosa += 1
        if result_species[:, 1][i] == 'ScinaxRuber':
            ScinaxRuber += 1

AdenomeraAndre,AdenomeraHylaedactylus,Ameeregatrivittata,HylaMinuta,HypsiboasCinerascens,HypsiboasCordobae,LeptodactylusFuscus,OsteocephalusOophagus,Rhinellagranulosa,ScinaxRuber,total

(0, 0, 49, 0, 0, 0, 0, 0, 0, 0, 49)